In [1]:
import numpy as np

In [190]:
def integrate_adaptive(fun, a, b, tol, extra = None):
    
    print('Integrating between', a, b)
    
    if type(extra) == type(None): 
        
        x = np.linspace(a,b,5)
        y = fun(x)
        
        dx=(b-a)/(len(x)-1)
        area1=2*dx*(y[0]+4*y[2]+y[4])/3 #coarse step
        area2=dx*(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/3 #finer step
        err=np.abs(area1-area2)
        
        saved_data = np.vstack((x,y))
        
        global function_call_counter
        function_call_counter+=len(x)

        
        if err<tol: 
            return area2
        else: 
            midpoint = (a + b)/2
            left = integrate_adaptive(fun, a, midpoint, tol/2, extra = saved_data)
            right = integrate_adaptive(fun, midpoint, b, tol/2, extra = saved_data)
            
            return left + right
    
    else: 
        
        saved_data = extra
        saved_x = saved_data[0]
        saved_y = saved_data[1]

        
        x = np.linspace(a,b,5)
        y = np.empty(len(x))
        
        for i, x_i in enumerate(x):
            
            if x_i in saved_x:
                
                
                index = np.where(saved_x == x_i)[0][0] #Finds location of x_i in saved x values
                y[i] = saved_y[index] #Retrieve already calculated y value corresponding to x_i
            
            else: 
                
                saved_x = np.append(saved_x, x_i) #Save the x value for future reference
                y[i] = fun(x_i) #Calculate the new y value
                saved_y = np.append(saved_y, y[i]) #Save the y value for future reference
               
                function_call_counter+=1
            
                

        saved_data = np.vstack((saved_x, saved_y))
        
        #Now that we've got our x and y values, we can compute error as normal
        dx=(b-a)/(len(x)-1)
        area1=2*dx*(y[0]+4*y[2]+y[4])/3 #coarse step
        area2=dx*(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/3 #finer step
        err=np.abs(area1-area2)
        
        if err < tol: 
            return area2
        
        
        else:
            midpoint = (a + b)/2
            left = integrate_adaptive(fun, a, midpoint, tol/2, extra = saved_data)
            right = integrate_adaptive(fun, midpoint, b, tol/2, extra = saved_data)

            return left + right


In [191]:
function_call_counter = 0
a = 0
b = 1
ans = integrate_adaptive(np.exp, a, b, 1e-7, extra = None)

Integrating between 0 1
Integrating between 0 0.5
Integrating between 0 0.25
Integrating between 0 0.125
Integrating between 0.125 0.25
Integrating between 0.25 0.5
Integrating between 0.25 0.375
Integrating between 0.25 0.3125
Integrating between 0.3125 0.375
Integrating between 0.375 0.5
Integrating between 0.375 0.4375
Integrating between 0.4375 0.5
Integrating between 0.5 1
Integrating between 0.5 0.75
Integrating between 0.5 0.625
Integrating between 0.5 0.5625
Integrating between 0.5625 0.625
Integrating between 0.625 0.75
Integrating between 0.625 0.6875
Integrating between 0.6875 0.75
Integrating between 0.75 1
Integrating between 0.75 0.875
Integrating between 0.75 0.8125
Integrating between 0.8125 0.875
Integrating between 0.875 1
Integrating between 0.875 0.9375
Integrating between 0.9375 1


In [195]:
def integrate_adaptive_old(fun,x0,x1,tol):
    print('integrating between ',x0,x1)
    #hardwire to use simpsons
    x=np.linspace(x0,x1,5)
    y=fun(x)
    
    global function_call_counter_old
    function_call_counter_old+=5
    
    dx=(x1-x0)/(len(x)-1)
    area1=2*dx*(y[0]+4*y[2]+y[4])/3 #coarse step
    area2=dx*(y[0]+4*y[1]+2*y[2]+4*y[3]+y[4])/3 #finer step
    err=np.abs(area1-area2)
    if err<tol:
        return area2
    else:
        xmid=(x0+x1)/2
        left=integrate_adaptive_old(fun,x0,xmid,tol/2)
        right=integrate_adaptive_old(fun,xmid,x1,tol/2)
        return left+right


x0=0
x1=1
if True:
    ans=integrate_adaptive_old(np.exp,x0,x1,1e-7)
    print(ans-(np.exp(x1)-np.exp(x0)))
# else:
#     ans=integrate_adaptive(lorentz,x0,x1,1e-7)
#     print(ans-(np.arctan(x1)-np.arctan(x0)))

    

integrating between  0 1
integrating between  0 0.5
integrating between  0 0.25
integrating between  0 0.125
integrating between  0.125 0.25
integrating between  0.25 0.5
integrating between  0.25 0.375
integrating between  0.25 0.3125
integrating between  0.3125 0.375
integrating between  0.375 0.5
integrating between  0.375 0.4375
integrating between  0.4375 0.5
integrating between  0.5 1
integrating between  0.5 0.75
integrating between  0.5 0.625
integrating between  0.5 0.5625
integrating between  0.5625 0.625
integrating between  0.625 0.75
integrating between  0.625 0.6875
integrating between  0.6875 0.75
integrating between  0.75 1
integrating between  0.75 0.875
integrating between  0.75 0.8125
integrating between  0.8125 0.875
integrating between  0.875 1
integrating between  0.875 0.9375
integrating between  0.9375 1
1.9795676209355406e-09


In [194]:
function_call_counter_old = 0

In [196]:
function_call_counter_old

135